# Generate filter calibration curves
This script runs the monochromator through a series of wavelength steps, taking flats at each step to provide filter transmission curves for each filter.

In [ ]:
import sys
import asyncio
import time
import os
import numpy as np
from lsst.ts import salobj
from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS

## First, connect and enable the ATCalSys CSCs
### We do this first to let the lamp warm up while the telescope and dome are moving into position.

In [ ]:
domain = salobj.Domain()
atmonochromator = salobj.Remote(domain, 'ATMonochromator')
WhiteLightSource = salobj.Remote(name='ATWhiteLight', domain=domain)

## ATWhiteLight

In [ ]:
await WhiteLightSource.cmd_setLogLevel.set_start(level=10)
state = salobj.State.ENABLED
await salobj.set_summary_state(WhiteLightSource, state)

In [ ]:
# Start the chiller
await WhiteLightSource.cmd_setChillerTemperature.set_start(temperature=20)
await WhiteLightSource.cmd_startChiller.set_start()
# Confirm that the chiller has started and running at 20C

In [ ]:
# Open the shutter
await WhiteLightSource.cmd_openShutter.set_start()

In [ ]:
#Turn on lamp. It will then go into a warm up period before it will turn on 
await WhiteLightSource.cmd_turnLampOn.set_start(power = 910)

### If the ATWhiteLight faults and the lamp fails to come on, do the following:
(1) Use LOVE to cycle the CSC to standby->start->enable \
(2) Try again to turn the lamp on.

## ATMonochromator

In [ ]:
await atmonochromator.cmd_start.start()
state = salobj.State.ENABLED
await salobj.set_summary_state(atmonochromator, state, timeout=60)

In [ ]:
# Set the grating.  We will use grating 2, which is a mirror to pass white light
grating=2 # which is which? 1 appears to be red, 0 appears to be blue, 2 is the mirror
# takes ~40s from 0->1, (timed)
# takes ~180s from 1->0 (guess)
# takes 0->2 ~115s (measured)
# 2->0 takes 60s (measured)  -remeasured at 36s
# 2->1 takes 69s (measured)
# 1->2 takes 35s 
# Use grating 0 from 300-550 nm
# Use grating 1 from 550-115 0nm
await atmonochromator.cmd_selectGrating.set_start(gratingType=grating, timeout=180)

In [ ]:
# Set wavelength = 0 for white light
wave=0
await atmonochromator.cmd_changeWavelength.set_start(wavelength=wave)

In [ ]:
# Set slits wide open
entry_width=5.0
exit_width=5.0
await atmonochromator.cmd_changeSlitWidth.set_start(slit=1, slitWidth=entry_width)
await atmonochromator.cmd_changeSlitWidth.set_start(slit=2, slitWidth=exit_width)

In [ ]:
# Now check the state
async def get_params():
    tmp1 = await atmonochromator.evt_wavelength.aget()
    tmp2 = await atmonochromator.evt_entrySlitWidth.aget()
    tmp3 = await atmonochromator.evt_exitSlitWidth.aget()
    tmp4 = await atmonochromator.evt_selectedGrating.aget()
    return (tmp1.wavelength, tmp2.width, tmp3.width, tmp4.gratingType)

await get_params()

### You should see white light coming out of the flat illuminator.
### If the ATMonochromator faults or fails to respond, do the following:
(1) Put the ATMonochromator CSC in STANDBY \
(2) Open the plastic cover on the right side of the illumination system. \
(3) Reach inside with the long metal bar on top of the illuminator and \
power cycle the illuminator by turning the rocker switch off and back on. \
(4) Initialize the monochromator using  Microsoft Remote Desktop as explained \
in Section 4.6 of https://tstn-032.lsst.io/ \
(5) Re-start and enable the CSC using the cells above or LOVE

### Ticket DM-38693 hopefully will eliminate the need to do this.

## Then, enable LATISS and ATCS and position telescope

In [ ]:
atcs = ATCS(domain)
latiss = LATISS(domain)
await asyncio.gather(atcs.start_task, latiss.start_task)

In [ ]:
await latiss.enable()

In [ ]:
await atcs.enable()

In [ ]:
# This will position the telescope and dome for the old illuminator
await atcs.prepare_for_flatfield()

In [ ]:
await atcs.disable_dome_following()

In [ ]:
# This is for the new dome flat illuminator
await atcs.slew_dome_to(355.0)

In [ ]:
# This is for the new dome flat illuminator
start_az=181.7
start_el=39.0
start_rot=0
await atcs.point_azel(az=start_az, el=start_el, rot_tel=start_rot)

## Now we are ready to take the calibrations
### Please verify that the flat level is between 10,000 and 40,000 ADU on RubinTV

In [ ]:
# Take 25 biases
for i in range(25):
    await latiss.take_bias(nbias=1)
    await asyncio.sleep(2.0)

In [ ]:
# Take darks with a range of exposure times:
await latiss.take_darks(exptime=5.0, ndarks=5)
await latiss.take_darks(exptime=15.0, ndarks=2)
await latiss.take_darks(exptime=30.0, ndarks=15)

In [ ]:
# Take 1 flat with each of the 3 filters.  Verify that the flats are between 10,000-40,000 ADU
# Adjust exptimes if necessary
await latiss.take_flats(exptime=9.0, nflats=1, filter='SDSSi_65mm', grating='empty_1')
await latiss.take_flats(exptime=9.0, nflats=1, filter='SDSSr_65mm', grating='empty_1')
await latiss.take_flats(exptime=9.0, nflats=1, filter='SDSSg_65mm', grating='empty_1')

In [ ]:
# Now take 20 flats with all 3 filters
await latiss.take_flats(exptime=9.0, nflats=20, filter='SDSSi_65mm', grating='empty_1')
await latiss.take_flats(exptime=9.0, nflats=20, filter='SDSSr_65mm', grating='empty_1')
await latiss.take_flats(exptime=9.0, nflats=20, filter='SDSSg_65mm', grating='empty_1')

## Done.  Shutting down.
### After turning off the lamp, you need to wait 15 minutes before turning off the chiller

In [ ]:
# Turn off the lamp
tmp = await WhiteLightSource.cmd_turnLampOff.set_start()

In [ ]:
# Close the shutter
tmp = await WhiteLightSource.cmd_closeShutter.set_start()

In [ ]:
# Turn off the chiller
# If you try to turn it off before 15 minutes, it will give an error
# and tell you how much time is left before you can turn it off
tmp = await WhiteLightSource.cmd_stopChiller.start()

In [ ]:
# Put ATWhiteLight in STANDBY
# You can't do this until the chiller is off
state = salobj.State.STANDBY
await salobj.set_summary_state(WhiteLightSource, state)

In [ ]:
# Put ATMonochromator in STANDBY
state = salobj.State.STANDBY
await salobj.set_summary_state(atmonochromator, state, timeout=60)

In [ ]:
# Shutdown ATCS.  This will put ATCS in STANDBY and park the telescope and dome.
await atcs.shutdown()

In [ ]:
# Put LATISS in STANDBY
await latiss.standby()